In [1]:
# State

In [1]:
import pickle
import time
from typing import List
import datetime as dt
from pathlib import Path
import copy

import pandas as pd

import numpy as np
import pandas as pd
import plotly.graph_objects as go
import plotly.express as px

from tqdm import tqdm

from quara.data_analysis import data_analysis, physicality_violation_check, report
from quara.data_analysis.projected_gradient_descent_backtracking import (
    ProjectedGradientDescentBacktracking,
    ProjectedGradientDescentBacktrackingOption,
)
from quara.data_analysis.weighted_probability_based_squared_error import (
    WeightedProbabilityBasedSquaredError,
    WeightedProbabilityBasedSquaredErrorOption,
)
from quara.data_analysis.weighted_relative_entropy import (
    WeightedRelativeEntropy,
    WeightedRelativeEntropyOption,
)
from quara.objects.composite_system import CompositeSystem
from quara.objects.elemental_system import ElementalSystem
from quara.objects.matrix_basis import get_normalized_pauli_basis
from quara.objects.povm import (
    Povm,
    get_x_povm,
    get_y_povm,
    get_z_povm,
)
from quara.objects.qoperation import QOperation
from quara.objects.state import State, get_z0_1q, get_z1_1q, get_x0_1q
from quara.protocol.qtomography.standard.standard_qst import StandardQst
from quara.protocol.qtomography.standard.linear_estimator import LinearEstimator
from quara.protocol.qtomography.standard.loss_minimization_estimator import (
    LossMinimizationEstimator,
)
from quara.protocol.qtomography.standard.projected_linear_estimator import (
    ProjectedLinearEstimator,
)

from quara.data_analysis.simulation import StandardQTomographySimulationSetting
from quara.data_analysis.generation_setting import QOperationGenerationSetting, DepolarizedQOperationGenerationSetting
from quara.data_analysis.simulation_check import StandardQTomographySimulationCheck

from quara.settings import Settings
Settings.get_atol()

1e-13

In [2]:
%reload_ext autoreload
%autoreload 2

- test_settingのリストを作る
- 1つのtest_settingから、QOperationGenerationSettingのリストを作る
- QOperationGenerationSettingから、qoperationのサンプルをn_sample個作る
- qoperationのサンプルから、StandardQTomographySimulationSettingを作る
- 推定を実行する
- SimulationCheckで結果をチェックする
- 結果を格納する or 保存する 

In [8]:
from collections import namedtuple

In [9]:
# test_settingのリストを作る
e_sys = ElementalSystem(0, get_normalized_pauli_basis())
c_sys = CompositeSystem([e_sys])

# Base of Tester Object
povm_x = get_x_povm(c_sys)
povm_y = get_y_povm(c_sys)
povm_z = get_z_povm(c_sys)
tester_objects = [povm_x, povm_y, povm_z]

# Base of True Object
base_state =  get_z0_1q(c_sys)

In [10]:
# シミュレーションに必要な設定を作る
case_names = [
    "Linear (True)",
    "Linear (False)",
    "ProjectedLinear (True)",
    "ProjectedLinear (False)",
    "Maximum-Likelihood (True)",
    "Maximum-Likelihood (False)",
    "Least Squares (True)",
    "Least Squares (False)",
]

parametrizations = [True, False, True, False, True, False, True, False]

estimators = [LinearEstimator(), 
    LinearEstimator(),
    ProjectedLinearEstimator(),
    ProjectedLinearEstimator(),
    LossMinimizationEstimator(),
    LossMinimizationEstimator(),
    LossMinimizationEstimator(),
    LossMinimizationEstimator(),]

loss_list = [
    (None, None),
    (None, None),
    (None, None),
    (None, None),
    (WeightedRelativeEntropy(3),WeightedRelativeEntropyOption("identity")),
    (WeightedRelativeEntropy(4),WeightedRelativeEntropyOption("identity")),
    (WeightedProbabilityBasedSquaredError(3), WeightedProbabilityBasedSquaredErrorOption("identity")),
    (WeightedProbabilityBasedSquaredError(4),WeightedProbabilityBasedSquaredErrorOption("identity")),
]

def generate_pgdb_algo_option():
    return ProjectedGradientDescentBacktrackingOption(
        mode_stopping_criterion_gradient_descent="sum_absolute_difference_variable",
        num_history_stopping_criterion_gradient_descent=1
    )

algo_list = [
    (None, None),
    (None, None),
    (None, None),
    (None, None),
    (ProjectedGradientDescentBacktracking(), generate_pgdb_algo_option()),
    (ProjectedGradientDescentBacktracking(), generate_pgdb_algo_option()),
    (ProjectedGradientDescentBacktracking(), generate_pgdb_algo_option()),
    (ProjectedGradientDescentBacktracking(), generate_pgdb_algo_option())
]

In [11]:
NoiseSetting = namedtuple("NoiseSetting", ["qoperation_base", "method", "para"])
TestSetting = namedtuple(
    "TestSetting",
    [
        "true_object",
        "tester_objects",
        "seed",
        "n_rep",
        "num_data",
        "n_sample",
        "schedules",
        "case_names",
        "estimators",
        "algo_list",
        "loss_list",
        "parametrizations"
    ],
)

Result = namedtuple(
    "Result",
    ["result_index", "simulation_setting", "estimation_results", "check_result"],
)

GenerationSettings = namedtuple(
    "GenerationSettings", ["true_setting", "tester_settings"]
)


In [12]:
noise_setting_0 = NoiseSetting(
    qoperation_base=base_state, method="depolarized", para={"error_rate": 0.1}
)
noise_setting_1 = NoiseSetting(
    qoperation_base=povm_x, method="depolarized", para={"error_rate": 0.1}
)
noise_setting_2 = NoiseSetting(
    qoperation_base=povm_y, method="depolarized", para={"error_rate": 0.1}
)
noise_setting_3 = NoiseSetting(
    qoperation_base=povm_z, method="depolarized", para={"error_rate": 0.1}
)

test_setting_0 = TestSetting(
    true_object=noise_setting_0,
    tester_objects=[noise_setting_1, noise_setting_2, noise_setting_3],
    seed=777,
    n_sample=2,
    n_rep=10,
    num_data=[100, 1000],
    schedules="all",
    case_names=case_names,
    estimators=estimators,
    algo_list=algo_list,
    loss_list=loss_list,
    parametrizations=parametrizations
)

# TODO: 複数のtest_settingをtest_settingsに格納する
test_settings = [test_setting_0, test_setting_0]

In [13]:
# 1つのtest_settingから、QOperationGenerationSettingのリストを作る
def to_generation_setting(noise_setting):
    # TODO: 他に良い方法がないか検討
    if noise_setting.method == "depolarized":
        target_class =  DepolarizedQOperationGenerationSetting
    else:
        message = f"noise_setting.method='{noise_setting.method}' is not implemented."
        raise NotImplementedError(message)
    return target_class(
            qoperation_base=noise_setting.qoperation_base,
            c_sys=noise_setting.qoperation_base.composite_system,
            **noise_setting.para
        )

def to_generation_settings(test_setting):
    true_setting = to_generation_setting(test_setting.true_object)
    tester_settings = [to_generation_setting(setting) for setting in test_setting.tester_objects]
    generation_settings = GenerationSettings(true_setting=true_setting, tester_settings=tester_settings)
    return generation_settings

def to_simulation_setting(test_setting, true_object, tester_objects, case_index: int) -> StandardQTomographySimulationSetting:
    return StandardQTomographySimulationSetting(
        name=test_setting.case_names[case_index],
        estimator=test_setting.estimators[case_index],
        loss=test_setting.loss_list[case_index][0],
        loss_option=test_setting.loss_list[case_index][1],
        algo=test_setting.algo_list[case_index][0],
        algo_option=test_setting.algo_list[case_index][1],
        true_object=true_object,
        tester_objects=tester_objects,
        n_rep=test_setting.n_rep,
        seed=test_setting.seed,
        num_data = test_setting.num_data,
        schedules= test_setting.schedules
    )

In [14]:
from quara.protocol.qtomography.standard.standard_qst import StandardQst
from quara.protocol.qtomography.standard.standard_qpt import StandardQpt
from quara.protocol.qtomography.standard.standard_povmt import StandardPovmt


def generate_qtomography(sim_setting, para: bool, eps_proj_physical: float) -> "StandardQTomography":
    true_object = sim_setting.true_object
    tester_objects = sim_setting.tester_objects
    seed = sim_setting.seed

    if type(true_object) == State:
        return StandardQst(
            tester_objects,
            on_para_eq_constraint=para,
            seed=seed,
            eps_proj_physical=eps_proj_physical,
        )
    if type(true_object) == Povm:
        return StandardPovmt(
            tester_objects,
            on_para_eq_constraint=para,
            seed=seed,
            eps_proj_physical=eps_proj_physical,
        )
    if type(true_object) == Gate:
        states = [t for t in tester_objects if type(t) == State]
        povms = [t for t in tester_objects if type(t) == Povm]

        return StandardQpt(
            states=states,
            povms=povms,
            on_para_eq_constraint=para,
            seed=seed,
            eps_proj_physical=eps_proj_physical,
        )
    # TODO: message
    return TypeError()

In [24]:
def result2dict(result):
    result_dict = dict(
        test_setting_index=result.result_index["test_setting_index"],
        sample_index=result.result_index["sample_index"],
        case_index=result.result_index["case_index"],
        name=result.simulation_setting.name,
        total_result=result.check_result["total_result"])

    def _make_warning_text(r):
        possibly_ok = r["detail"]["possibly_ok"]
        to_be_checked = r["detail"]["to_be_checked"]
        warning_text = ""
        if not possibly_ok:
            warning_text = f"Consistency: possibly_ok={possibly_ok}, to_be_checked={to_be_checked}"
        return warning_text
    
    check_result = {}
    warning_text = ""
    
    for r in result.check_result["results"]:
        check_result[r["name"]] = r["result"]
        if r["name"] == "Consistency":
            warning_text +=  _make_warning_text(r)

    result_dict.update(check_result)
    result_dict["warning"] = warning_text
        
    return result_dict

In [25]:
def write_results(results: List[Result], dir_path) -> None:
    dir_path = Path(dir_path)
    dir_path.mkdir(parents=True, exist_ok=True)
    path = dir_path / "check_result.csv"

    result_dict_list = [result2dict(r) for r in results]
    sample_result_df = pd.DataFrame(result_dict_list)
    sample_result_df.to_csv(path, index=None)

    print(f"Completed to write csv. {path}")


def write_result_case_unit(result: namedtuple, root_dir: str) -> None:
    test_setting_index = result.result_index["test_setting_index"]
    sample_index = result.result_index["sample_index"]
    case_index = result.result_index["case_index"]
    
    # Save all
    dir_path = Path(f"{root_dir}/{test_setting_index}/{sample_index}")
    dir_path.mkdir(parents=True, exist_ok=True)
    path = dir_path / f"case_{case_index}_result.pickle"
    with open(path, 'wb') as f:
        pickle.dump(result, f)

    check_result = result.check_result
    path = dir_path / f"case_{case_index}_check_result.json"
    with open(path, "w") as f:
        json.dump(check_result, f, ensure_ascii=False, indent=4, separators=(',', ': '))

    
def write_result_sample_unit(results: List[namedtuple], root_dir: str):
    test_setting_index = results[0].result_index["test_setting_index"]
    sample_index = results[0].result_index["sample_index"]
    dir_path = Path(root_dir) / str(test_setting_index) / str(sample_index)

    write_results(results, dir_path)


def write_result_test_setting_unit(results: List[namedtuple], root_dir: str):
    test_setting_index = results[0].result_index["test_setting_index"]
    dir_path = Path(root_dir) / str(test_setting_index)
    
    write_results(results, dir_path)
    
def write_pdf_report(results, root_dir):
    test_setting_index = results[0].result_index["test_setting_index"]
    sample_index = results[0].result_index["sample_index"]
    
    dir_path = Path(root_dir) / str(test_setting_index) / str(sample_index)
    dir_path.mkdir(parents=True, exist_ok=True)
    path = dir_path / f"{test_setting_index}_{sample_index}_quara_report.pdf"
    
    estimation_results_list = [r.estimation_results for r in results]
    sim_settings = [r.simulation_setting for r in results]
    
    report.export_report(path, estimation_results_list, sim_settings)

In [31]:
def execute_simulation_case_unit(
    test_setting,
    true_object,
    tester_objects,
    case_index: int,
    sample_index: int,
    test_setting_index: int,
    root_dir: str,
) -> Result:
    # QTomographySimulationSettingを作る
    sim_setting = to_simulation_setting(
        test_setting, true_object, tester_objects, case_index
    )
    print(f"Case {case_index}: {sim_setting.name}")

    def _copy_sim_setting(source):
        return StandardQTomographySimulationSetting(
            name=source.name,
            true_object=source.true_object,
            tester_objects=source.tester_objects,
            estimator=source.estimator,
            loss=copy.deepcopy(source.loss),
            loss_option=source.loss_option,
            algo=copy.deepcopy(source.algo),
            algo_option=source.algo_option,
            seed=source.seed,
            n_rep=source.n_rep,
            num_data=source.num_data,
            schedules=source.schedules
        )

    org_sim_setting = _copy_sim_setting(sim_setting)

    # QTomographyの作成
    qtomography = generate_qtomography(
        sim_setting,
        para=test_setting.parametrizations[case_index],
        eps_proj_physical=1e-13,
    )

    # 推定
    estimation_results = data_analysis.execute_simulation(
        qtomography=qtomography, simulation_setting=sim_setting
    )

    # 自動テスト
    sim_check = StandardQTomographySimulationCheck(sim_setting, estimation_results)
    check_result = sim_check.execute_all(show_detail=False, with_detail=True)

    # 結果の表示
    if not check_result["total_result"]:
        start_red = "\033[31m"
        end_color = "\033[0m"
        print(f"Total Result: {start_red}NG{end_color}")

    result_index = dict(
        test_setting_index=test_setting_index,
        sample_index=sample_index,
        case_index=case_index,
    )

    # 結果の格納
    result = Result(
        result_index=result_index,
        simulation_setting=org_sim_setting,
        estimation_results=estimation_results,
        check_result=check_result,
    )
    # Save
    write_result_case_unit(result, root_dir=root_dir)
    return result

In [28]:
def execute_simulation_sample_unit(
    test_setting, generation_settings, test_setting_index, sample_index, root_dir, pdf_mode: str="only_ng"
) -> List[Result]:
    # sampleの生成
    true_object = generation_settings.true_setting.generate()
    tester_objects = [
        tester_setting.generate()
        for tester_setting in generation_settings.tester_settings
    ]
    results = []
    case_n = len(test_setting.case_names)

    for case_index in range(case_n):
        result = execute_simulation_case_unit(
            test_setting,
            true_object=true_object,
            tester_objects=tester_objects,
            case_index=case_index,
            sample_index=sample_index,
            test_setting_index=test_setting_index,
            root_dir=root_dir,
        )
        results.append(result)

    # Save
    write_result_sample_unit(results, root_dir=root_dir)
    
    # Save PDF
    if pdf_mode == "all":
        write_pdf_report(results, root_dir)
    elif pdf_mode == "only_ng":
        total_results = [r.check_result["total_result"] for r in results]
        print(f"{total_results=}")
        print(np.all(total_results))
        if not np.all(total_results):
            write_pdf_report(results, root_dir)
    elif pdf_mode == "none":
        pass
    else:
        message = "`pdf_mode` must be 'all', 'only_ng', or 'none'."
        raise ValueError(message)
        
    return results


def execute_simulation_test_setting_unit(test_setting, dir_path, pdf_mode: str='only_ng') -> List[Result]:
    generation_settings = to_generation_settings(test_setting)
    n_sample = test_setting.n_sample
    results = []

    for sample_index in range(n_sample):
        sample_results = execute_simulation_sample_unit(
            test_setting,
            generation_settings,
            test_setting_index,
            sample_index,
            root_dir,
            pdf_mode=pdf_mode
        )
        results += sample_results

    # 保存（test_index単位）
    write_result_test_setting_unit(results, dir_path)
    return results

In [54]:
def print_summary(results, elapsed_time):
    result_dict_list = [result2dict(r) for r in results]
    result_df = pd.DataFrame(result_dict_list)
    case_n = result_df.shape[0]
    ok_n = result_df[result_df["total_result"]].shape[0]
    ng_n = result_df[~result_df["total_result"]].shape[0]
    warning_n = result_df[result_df["total_result"] & result_df["warning"].isnull()].shape[0]
    
    def _to_h_m_s(sec) -> tuple:
        m, s = divmod(int(sec), 60)
        h, m = divmod(m, 60)
        return h, m, s
    
    h, m, s = _to_h_m_s(elapsed_time)
    time_text = '{:.1f}s '.format(elapsed_time)
    time_text += f"({h}:{str(m).zfill(2)}:{str(s).zfill(2)})"

    start_red = "\033[31m"
    start_green = '\033[32m'
    start_yellow = '\033[33m'
    end_color = "\033[0m"

    summary_text = f"{start_yellow}=============={end_color} "
    summary_text += f"{start_green}OK: {ok_n} cases{end_color} ({start_yellow}{warning_n} warnings{end_color}), "
    summary_text += f"{start_red}NG: {ng_n} cases{end_color} "
    summary_text += f"{start_yellow} in {time_text}s=============={end_color}"

    print(summary_text)

In [29]:
root_dir = "result"
all_results = []
start = time.time()

for test_setting_index, test_setting in enumerate(test_settings):
    test_results= execute_simulation_test_setting_unit(test_setting, root_dir, pdf_mode="only_ng")
    all_results += test_results

# Save
write_results(all_results, root_dir)

elapsed_time = time.time() - start
print_summary(all_results, elapsed_time)

  0%|          | 0/10 [00:00<?, ?it/s]

Case 0: Linear (True)
=======schedules=======
('state', 0): ('povm', 0)
('state', 0): ('povm', 1)
('state', 0): ('povm', 2)
=======schedules=======
('state', 0): ('povm', 0)
('state', 0): ('povm', 1)
('state', 0): ('povm', 2)
=======schedules=======
('state', 0): ('povm', 0)
('state', 0): ('povm', 1)
('state', 0): ('povm', 2)
=======schedules=======
('state', 0): ('povm', 0)
('state', 0): ('povm', 1)
('state', 0): ('povm', 2)
=======schedules=======
('state', 0): ('povm', 0)
('state', 0): ('povm', 1)
('state', 0): ('povm', 2)
=======schedules=======
('state', 0): ('povm', 0)
('state', 0): ('povm', 1)
('state', 0): ('povm', 2)
=======schedules=======
('state', 0): ('povm', 0)
('state', 0): ('povm', 1)
('state', 0): ('povm', 2)


100%|██████████| 10/10 [00:00<00:00, 7741.42it/s]

=======schedules=======
('state', 0): ('povm', 0)
('state', 0): ('povm', 1)
('state', 0): ('povm', 2)
=======schedules=======
('state', 0): ('povm', 0)
('state', 0): ('povm', 1)
('state', 0): ('povm', 2)
=======schedules=======
('state', 0): ('povm', 0)
('state', 0): ('povm', 1)
('state', 0): ('povm', 2)
=======schedules=======
('state', 0): ('povm', 0)
('state', 0): ('povm', 1)
('state', 0): ('povm', 2)


========== Summary ============
Name: Linear (True)
MSE of Empirical Distributions: OK
Consistency: OK (to_be_checked=False)
MSE of estimators: OK
Physicality Violation: OK


...

100%|██████████| 10/10 [00:00<00:00, 18649.64it/s]

​​Generating MSE of empirical distributions blocks ...


​​Generating consictency test blocks ...
=======schedules=======
('state', 0): ('povm', 0)
('state', 0): ('povm', 1)
('state', 0): ('povm', 2)
=======schedules=======
('state', 0): ('povm', 0)
('state', 0): ('povm', 1)
('state', 0): ('povm', 2)
=======schedules=======
('state', 0): ('povm', 0)
('state', 0): ('povm', 1)
('state', 0): ('povm', 2)
=======schedules=======
('state', 0): ('povm', 0)
('state', 0): ('povm', 1)
('state', 0): ('povm', 2)
=======schedules=======
('state', 0): ('povm', 0)
('state', 0): ('povm', 1)
('state', 0): ('povm', 2)
=======schedules=======
('state', 0): ('povm', 0)
('state', 0): ('povm', 1)
('state', 0): ('povm', 2)
=======schedules=======
('state', 0): ('povm', 0)
('state', 0): ('povm', 1)
('state', 0): ('povm', 2)
=======schedules=======
('state', 0): ('povm', 0)
('state', 0): ('povm', 1)
('state', 0): ('povm', 2)
=======schedules=======
('state', 0): ('povm', 0)
('state', 0): ('povm', 1)
('state', 0): ('povm', 2)
=======schedules=======
('state', 0): ('p

100%|██████████| 10/10 [00:00<00:00, 3521.37it/s]


Converting to PDF report ...
​Deleting temporary files ...
Completed to export pdf. (result/1/1/1_1_quara_report.pdf)
Completed to write csv. result/1/check_result.csv
Completed to write csv. result/check_result.csv


In [55]:
start = time.time()
time.sleep(3)
elapsed_time = time.time() - start
print_summary(all_results, elapsed_time)

============== OK: 16 cases (0 warnings), NG: 16 cases  in 3.0s (0:00:03)s==============
